In [2]:
import os
from pathlib import Path
from typing import Union, Dict
from detect import run
from metrics import analyze_labels

def process_input(input_path: str, output_project: str = "results") -> Union[bool, Dict[str, float]]:
    """
    Анализирует изображения и видео через YOLO, возвращает метрики или False если объектов нет.
    
    Параметры:
        input_path (str): Путь к папке с изображениями/видео
        output_project (str): Папка для сохранения результатов
    
    Возвращает:
        dict или False: Словарь с метриками или False если объектов нет
    """
    # Запуск детекции
    run(
        weights="./weights/warn150322_last.pt",
        source=input_path,
        conf_thres=0.3,
        project=output_project,
        name="detection",
        save_txt=True,
        save_conf=True,
        line_thickness=5,
        exist_ok=True
    )

    # Путь к лейблам
    labels_dir = Path(output_project) / "detection" / "labels"
    
    # Проверка наличия объектов
    if not labels_dir.exists():
        return False
    
    stats = analyze_labels(str(labels_dir))
    
    if stats["global_stats"]["total_objects"] == 0:
        return False

    classes = stats["classes"]
    if not classes:
        return False

    # Расчет метрик
    total_objects = stats["global_stats"]["total_objects"]
    
    return {
        "Smax": max(c.max_s for c in classes),
        "Scom": stats["global_stats"]["total_coverage"],
        "Savg": stats["global_stats"]["total_coverage"] / total_objects,
        "Dx": sum(c.var_x for c in classes) / len(classes),
        "Dy": sum(c.var_y for c in classes) / len(classes),
        "Cmax": max(c.max_c for c in classes),
        "Cavg": sum(c.avg_c * c.count for c in classes) / total_objects,
        "n": total_objects
    }

# Пример использования
if __name__ == "__main__":
    result = process_input("data/images")
    print(result if result else "Объекты не обнаружены")

YOLOv5  v7.0-419-gcd44191c Python-3.11.9 torch-2.7.0+cpu CPU

Fusing layers... 
Model summary: 224 layers, 7070092 parameters, 0 gradients
image 1/4 D:\repos\content\project\api\yolov5\data\images\Class=FlafIchkeria_Pic=19.png: 640x640 1 Ichkeria, 240.2ms
image 2/4 D:\repos\content\project\api\yolov5\data\images\Class=FlafIgil_Pic=1.png: 640x640 1 Igil, 207.7ms
image 3/4 D:\repos\content\project\api\yolov5\data\images\Class=FlafIgil_Pic=5.png: 640x640 1 Igil, 165.0ms
image 4/4 D:\repos\content\project\api\yolov5\data\images\Class=FlafTaliban_Pic=10.png: 640x640 3 Talibans, 181.7ms
Speed: 1.3ms pre-process, 198.7ms inference, 1.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to results\detection
4 labels saved to results\detection\labels


{'Smax': 0.47546371875, 'Scom': 6.437839130859999, 'Savg': 0.2145946376953333, 'Dx': 0.022322702600000004, 'Dy': 0.0036393408605185193, 'Cmax': 0.965966, 'Cavg': 0.9354493333333334, 'n': 30}
